# Foooooooooooocus
#### Kaggle NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Kaggle](https://www.kaggle.com) 部署 Fooocus 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式。
2. 安装 Fooocus：根据环境配置安装 Fooocus 和运行环境。
3. 下载模型：下载可选列表中的模型。
4. 启动 Fooocus：启动 Fooocus，并显示访问地址。

## 提示
1. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
2. 已知 CloudFlare、Gradio 内网穿透会导致 [Kaggle](https://www.kaggle.com) 平台强制关机，在使用 [Kaggle](https://www.kaggle.com) 平台时请勿勾选这两个选项。
3. 请不要生成违禁图片，这将导致 Kaggle 账号被永久封禁！

### 参数配置
1. [[下一个单元 →](#安装-Fooocus)]

In [ ]:
# 消息格式输出
def echo(msg):
    print(f":: {msg}")



# ARIA2
class ARIA2:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载器
    def aria2(self, url, path, filename):
        import os
        if not os.path.exists(path + "/" + filename):
            echo(f"开始下载 {filename} ，路径: {path}/{filename}")
            !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
            if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"{filename} 下载完成")
            else:
                echo(f"{filename} 下载中断")
        else:
            if os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"开始下载 {filename} ，路径: {path}/{filename}")
                !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
                if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                    echo(f"{filename} 下载完成")
                else:
                    echo(f"{filename} 下载中断")
            else:
                echo(f"{filename} 文件已存在，路径: {path}/{filename}")


    # 大模型下载
    def get_sd_model(self, url, filename):
        pass


    # lora下载
    def get_lora_model(self, url, filename):
        pass



# GIT
class GIT:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 检测要克隆的项目是否存在于指定路径
    def exists(self, addr=None, path=None, name=None):
        import os
        if addr is not None:
            if path is None and name is None:
                path = os.getcwd() + "/" + addr.split("/").pop().split(".git", 1)[0]
            elif path is None and name is not None:
                path = os.getcwd() + "/" + name
            elif path is not None and name is None:
                path = os.path.normpath(path) + "/" + addr.split("/").pop().split(".git", 1)[0]

        if os.path.exists(path):
            return True
        else:
            return False


    # 克隆项目
    def clone(self, addr, path=None, name=None):
        import os
        repo = addr.split("/").pop().split(".git", 1)[0]
        if not self.exists(addr, path, name):
            echo(f"开始下载 {repo}")
            if path is None and name is None:
                path = os.getcwd()
                name = repo
            elif path is not None and name is None:
                name = repo
            elif path is None and name is not None:
                path = os.getcwd()
            !git clone {addr} "{path}/{name}" --recurse-submodules
        else:
            echo(f"{repo} 已存在")



# TUNNEL
class TUNNEL:
    LOCALHOST_RUN = "localhost.run"
    REMOTE_MOE = "remote.moe"
    WORKSPACE = ""
    WORKFOLDER = ""
    PORT = ""


    def __init__(self, workspace, workfolder, port) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.PORT = port


    # ngrok内网穿透
    def ngrok(self, ngrok_token: str):
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        port = self.PORT
        ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            return ssh_tunnel.public_url
        else:
            return ssh_tunnels[0].public_url


    # cloudflare内网穿透
    def cloudflare(self):
        from pycloudflared import try_cloudflare
        port = self.PORT
        urls = try_cloudflare(port).tunnel
        return urls


    from typing import Union
    from pathlib import Path

    # 生成ssh密钥
    def gen_key(self, path: Union[str, Path]) -> None:
        import subprocess
        import shlex
        from pathlib import Path
        path = Path(path)
        arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
        args = shlex.split(arg_string)
        subprocess.run(args, check=True)
        path.chmod(0o600)


    # ssh内网穿透
    def ssh_tunnel(self, host: str) -> None:
        import subprocess
        import atexit
        import shlex
        import re
        import os
        from pathlib import Path
        from tempfile import TemporaryDirectory

        ssh_name = "id_rsa"
        ssh_path = Path(self.WORKSPACE) / ssh_name
        port = self.PORT

        tmp = None
        if not ssh_path.exists():
            try:
                self.gen_key(ssh_path)
            # write permission error or etc
            except subprocess.CalledProcessError:
                tmp = TemporaryDirectory()
                ssh_path = Path(tmp.name) / ssh_name
                self.gen_key(ssh_path)

        arg_string = f"ssh -R 80:127.0.0.1:{port} -o StrictHostKeyChecking=no -i {ssh_path.as_posix()} {host}"
        args = shlex.split(arg_string)

        tunnel = subprocess.Popen(
            args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)
        if tmp is not None:
            atexit.register(tmp.cleanup)

        tunnel_url = ""
        LOCALHOST_RUN = self.LOCALHOST_RUN
        lines = 27 if host == LOCALHOST_RUN else 5
        localhostrun_pattern = re.compile(r"(?P<url>https?://\S+\.lhr\.life)")
        remotemoe_pattern = re.compile(r"(?P<url>https?://\S+\.remote\.moe)")
        pattern = localhostrun_pattern if host == LOCALHOST_RUN else remotemoe_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")

            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                if lines == 27:
                    os.environ['LOCALHOST_RUN'] = tunnel_url
                    return tunnel_url
                else:
                    os.environ['REMOTE_MOE'] = tunnel_url
                    return tunnel_url
                # break
        else:
            echo(f"启动 {host} 内网穿透失败")


    # localhost.run穿透
    def localhost_run(self):
        urls = self.ssh_tunnel(self.LOCALHOST_RUN)
        return urls


    # remote.moe内网穿透
    def remote_moe(self):
        urls = self.ssh_tunnel(self.REMOTE_MOE)
        return urls


    # gradio内网穿透
    def gradio(self):
        import subprocess
        import shlex
        import atexit
        import re
        port = self.PORT
        cmd = f"gradio-tunneling --port {port}"
        cmd = shlex.split(cmd)
        tunnel = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)

        tunnel_url = ""
        lines = 5
        gradio_pattern = re.compile(r"(?P<url>https?://\S+\.gradio\.live)")
        pattern = gradio_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")
            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                return tunnel_url
        else:
            echo(f"启动 Gradio 内网穿透失败")


    # 启动内网穿透
    def start(self, ngrok=False, ngrok_token=None, cloudflare=False, remote_moe=False, localhost_run=False, gradio=False):
        if cloudflare is True or ngrok is True or ngrok_token is not None or remote_moe is True or localhost_run is True or gradio is True:
            echo("启动内网穿透")

        if cloudflare is True:
            cloudflare_url = self.cloudflare()
        else:
            cloudflare_url = None

        if ngrok is True and ngrok_token is not None:
            ngrok_url = self.ngrok(ngrok_token)
        else:
            ngrok_url = None

        if remote_moe is True:
            remote_moe_url = self.remote_moe()
        else:
            remote_moe_url = None

        if localhost_run is True:
            localhost_run_url = self.localhost_run()
        else:
            localhost_run_url = None

        if gradio is True:
            gradio_url = self.gradio()
        else:
            gradio_url = None

        echo("下方为访问地址")
        print("==================================================================================")
        echo(f"CloudFlare: {cloudflare_url}")
        echo(f"Ngrok: {ngrok_url}")
        echo(f"remote.moe: {remote_moe_url}")
        echo(f"localhost_run: {localhost_run_url}")
        echo(f"Gradio: {gradio_url}")
        print("==================================================================================")




# ENV
class ENV:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 准备ipynb笔记自身功能的依赖
    def prepare_env_depend(self, use_mirror=True):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        echo("安装自身组件依赖")
        !pip install pyngrok pycloudflared gradio-tunneling {pip_mirror}
        !apt install aria2 ssh google-perftools -y


    # 安装pytorch和xformers
    def prepare_torch(self, torch_ver, xformers_ver, use_mirror=False):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        echo("安装 PyTorch")
        !pip install {torch_ver} {pip_mirror}
        echo("安装 xFormers")
        !pip install {xformers_ver} {pip_mirror}


    # python软件包安装
    # 可使用的操作:
    # 安装: install -> install
    # 仅安装: install_single -> install --no-deps
    # 强制重装: force_install -> install --force-reinstall
    # 仅强制重装: force_install_single -> install --force-reinstall --no-deps
    # 更新: update -> install --upgrade
    # 卸载: uninstall -y
    def py_pkg_manager(self, pkg, type=None, use_mirror=False):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        if type == "install":
            func = "install"
            args = ""
        elif type == "install_single":
            func = "install"
            args = "--no-deps"
        elif type == "force_install":
            func = "install"
            args = "--force-reinstall"
        elif type == "force_install_single":
            func = "install"
            args = "install --force-reinstall --no-deps"
        elif type == "update":
            func = "install"
            args = "--upgrade"
        elif type == "uninstall":
            func = "uninstall"
            args = "-y"
            pip_mirror = ""
        else:
            echo(f"未知操作: {type}")
            return
        echo(f"执行操作: pip {func} {pkg} {args} {pip_mirror}")
        !pip {func} {pkg} {args} {pip_mirror}


    # 安装requirements.txt依赖
    def install_requirements(self, path, use_mirror=False):
        import os
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"
        if os.path.exists(path):
            echo("安装依赖")
            !pip install -r "{path}" {pip_mirror}
        else:
            echo("依赖文件路径为空")


    # 配置内存优化
    def tcmalloc(self):
        echo("配置内存优化")
        import os
        os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4"



# MANAGER
class MANAGER:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 清理ipynb笔记的输出
    def clear_up(self, opt):
        from IPython.display import clear_output
        clear_output(wait=opt)


    # 检查gpu是否可用
    def check_gpu(self):
        echo("检测 GPU 是否可用")
        import tensorflow as tf
        echo(f"TensorFlow 版本: {tf.__version__}")
        if tf.test.gpu_device_name():
            echo("GPU 可用")
        else:
            echo("GPU 不可用")
            raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")


    # 配置google drive
    def config_google_drive(self):
        echo("挂载 Google Drive")
        import os
        if not os.path.exists('/content/drive/MyDrive'):
            from google.colab import drive
            drive.mount('/content/drive')
            echo("Google Dirve 挂载完成")
        else:
            echo("Google Drive 已挂载")

        # 检测并创建输出文件夹
        if os.path.exists('/content/drive/MyDrive'):
            if not os.path.exists('/content/drive/MyDrive/fooocus_output'):
                echo("在 Google Drive 创建 fooocus_ouput 文件夹")
                !mkdir -p /content/drive/MyDrive/fooocus_output
        else:
            raise Exception("未挂载 Google Drive，请重新挂载后重试！")



# FOOOCUS
class FOOOCUS(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 7865)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载大模型
    def get_sd_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/checkpoints"
        super().aria2(url, path, filename)


    # 下载lora模型
    def get_lora_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/loras"
        super().aria2(url, path, filename)


    # 下载配置文件
    def install_config(self):
        path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("下载配置文件")
        self.aria2("https://github.com/licyk/sd-webui-all-in-one/releases/download/archive/fooocus_config.json", path + "/presets", "custom.json")
        self.aria2("https://github.com/licyk/sd-webui-all-in-one/releases/download/archive/fooocus_path_config_kaggle.json", path, "config.txt")
        self.aria2("https://github.com/licyk/sd-webui-all-in-one/releases/download/archive/fooocus_zh_cn.json", path + "/language", "zh.json")


    # 安装fooocus
    def install(self, torch_ver, xformers_ver, use_mirror):
        import os
        self.check_gpu()
        self.prepare_env_depend(use_mirror)
        self.clone("https://github.com/lllyasviel/Fooocus", self.WORKSPACE)
        os.chdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        self.prepare_torch(torch_ver, xformers_ver, use_mirror)
        req_file = self.WORKSPACE + "/" + self.WORKFOLDER + "/requirements_versions.txt"
        self.install_requirements(req_file, use_mirror)
        self.py_pkg_manager("numpy", "force_install", use_mirror)
        self.install_config()
        self.tcmalloc()


#######################################################
try:    
    echo("尝试安装 ipywidgets 组件")
    !pip install ipywidgets -qq
    from IPython.display import clear_output
    clear_output(wait=False)
    INIT_CONFIG = 1
except:
    raise Exception("未初始化功能")

import ipywidgets as widgets


# PyTorch 版本
pytorch_ver = widgets.Textarea(value="torch==2.2.1+cu121 torchvision==0.17.1+cu121 torchaudio==2.2.1+cu121", placeholder="请填写 PyTorch 版本", description="PyTorch 版本: ", disabled=False)
xformers_ver = widgets.Textarea(value="xformers==0.0.25", placeholder="请填写 xFormers 版本", description="xFormers 版本: ", disabled=False)
# 内网穿透方式
use_remote_moe = widgets.Checkbox(value=True, description="使用 remote.moe 内网穿透", disabled=False)
use_localhost_run = widgets.Checkbox(value=True, description="使用 localhost.run 内网穿透", disabled=False) #@param {type:"boolean"}
use_cloudflare = widgets.Checkbox(value=False, description="使用 CloudFlare 内网穿透", disabled=False) #@param {type:"boolean"}
use_ngrok = widgets.Checkbox(value=False, description="使用 Ngrok 内网穿透", disabled=False) #@param {type:"boolean"}
ngrok_token = widgets.Textarea(value="", placeholder="请填写 Ngrok Token（可在 Ngrok 官网获取）", description="Ngrok Token: ", disabled=False)  #@param {type:"string"}
use_gradio = widgets.Checkbox(value=False, description="使用 Gradio 内网穿透", disabled=False) #@param {type:"boolean"}
# 模型下载
sd_xl_base = widgets.Checkbox(value=False, description="下载 sd_xl_base 模型", disabled=False)
kohakuXLGamma_rev1 = widgets.Checkbox(value=False, description="下载 kohakuXLGamma_rev1 模型", disabled=False)
kohakuXLBeta_beta7 = widgets.Checkbox(value=False, description="下载 kohakuXLBeta_beta7 模型", disabled=False)
Kohaku_XL_Delta = widgets.Checkbox(value=False, description="下载 Kohaku_XL_Delta 模型", disabled=False)
kohakuXL_Epsilon = widgets.Checkbox(value=True, description="下载 kohakuXL_Epsilon 模型", disabled=False)
animagine_XL_3 = widgets.Checkbox(value=False, description="下载 animagine_XL_3 模型", disabled=False)
animagine_XL_3_1 = widgets.Checkbox(value=True, description="下载 animagine_XL_3_1 模型", disabled=False)
heart_of_apple_xl = widgets.Checkbox(value=False, description="下载 heart_of_apple_xl 模型", disabled=False)
CounterfeitXL = widgets.Checkbox(value=False, description="下载 CounterfeitXL 模型", disabled=False)
bluePencilXL  = widgets.Checkbox(value=False, description="下载 bluePencilXL 模型", disabled=False)
animeIllustDiffusion = widgets.Checkbox(value=False, description="下载 animeIllustDiffusion 模型", disabled=False)
nekorayxl = widgets.Checkbox(value=False, description="下载 nekorayxl 模型", disabled=False)
anima_pencil_xl = widgets.Checkbox(value=False, description="下载 anima_pencil_xl 模型", disabled=False)
BArtstyleDB = widgets.Checkbox(value=False, description="下载 BArtstyleDB 模型", disabled=False)
pony_v6 = widgets.Checkbox(value=False, description="下载 pony_v6 模型", disabled=False)
# 自定义模型下载
model_url = widgets.Textarea(value="", placeholder="请填写模型下载链接", description="模型链接: ", disabled=False)  #@param {type:"string"}
model_name = widgets.Textarea(value="", placeholder="请填写模型名称，包括后缀名，例：kohaku-xl.safetensors", description="模型名称: ", disabled=False)  #@param {type:"string"}
model_type = widgets.Dropdown(options=[("Stable Diffusion 模型（大模型）", "sd"), ("LoRA 模型", "lora")], value="sd", description='模型种类: ')
# 创建实例
fooocus = FOOOCUS("/kaggle","Fooocus")
display(pytorch_ver, # PyTorch
        xformers_ver,
        use_remote_moe, # 内网穿透
        use_localhost_run,
        use_cloudflare,
        use_gradio,
        use_ngrok,
        ngrok_token,
        sd_xl_base, # 模型下载
        kohakuXLGamma_rev1,
        kohakuXLBeta_beta7,
        Kohaku_XL_Delta,
        kohakuXL_Epsilon,
        animagine_XL_3,
        animagine_XL_3_1,
        heart_of_apple_xl,
        CounterfeitXL,
        bluePencilXL,
        animeIllustDiffusion,
        nekorayxl,
        heart_of_apple_xl,
        anima_pencil_xl,
        BArtstyleDB,
        pony_v6,
        model_url, # 自定义模型下载
        model_name,
        model_type
        )

echo("配置参数完成")

### 安装 Fooocus
2. [[← 上一个单元](#参数配置)|[下一个单元 →](#启动-Fooocus)]

In [ ]:
try:    
    i = INIT_CONFIG
    INIT_CONFIG_1 = 1
except:
    raise Exception("未运行\"参数配置\"单元")

fooocus.install(pytorch_ver.value, xformers_ver.value, False)
fooocus.clear_up(False)
echo("Fooocus 运行环境配置完成")

# 预设模型下载
echo("下载模型中")
if sd_xl_base.value:
    fooocus.get_sd_model("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors", "sd_xl_base_1.0_0.9vae.safetensors")
if kohakuXLGamma_rev1.value:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors", "kohakuXLGamma_rev1.safetensors")
if kohakuXLBeta_beta7.value:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors", "kohakuXLBeta_beta7.safetensors")
if Kohaku_XL_Delta.value:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors", "kohaku-xl-delta-rev1.safetensors")
if kohakuXL_Epsilon.value:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors", "kohaku-xl-epsilon-rev1.safetensors")
if animagine_XL_3_1.value:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors", "animagine-xl-3.1.safetensors")
if animagine_XL_3.value:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors", "animagine-xl-3.0.safetensors")
if CounterfeitXL.value:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors", "CounterfeitXL-V1.0.safetensors")
if bluePencilXL.value:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323375", "bluePencilXL_v050.safetensors")
if animeIllustDiffusion.value:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors", "animeIllustDiffusion_v061.safetensors")
if nekorayxl.value:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors", "nekorayxl_v06W3.safetensors")
if heart_of_apple_xl.value:
    fooocus.get_sd_model("https://civitai.com/api/download/models/337306", "heart-of-apple-xl.safetensors")
if anima_pencil_xl.value:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323674", "anima_pencil_xl.safetensors")
if BArtstyleDB.value:
    fooocus.get_sd_model("https://civitai.com/api/download/models/299095", "BArtstyleDB_XL.safetensors")
if pony_v6.value:
    fooocus.get_sd_model("https://civitai.com/api/download/models/290640", "ponyDiffusionV6XL_v6StartWithThisOne.safetensors")

# 自定义模型下载
echo("下载自定义模型中")
if model_url.value != "" and model_name.value != "":
    if model_type.value == "sd":
        fooocus.get_sd_model(model_url.value, model_name.value)
    elif model_type.value == "lora":
        fooocus.get_lora_model(model_url.value, model_name.value)
else:
    echo("自定义模型下载未填写模型链接或者模型名称，请检查")
echo("模型下载完成")

### 启动 Fooocus
3. [[← 上一个单元](#安装-Fooocus)]

In [ ]:
try:    
    i = INIT_CONFIG_1
except:
    raise Exception("未运行\"安装\"单元")

echo("启动 Fooocus")
fooocus.tun.start(ngrok=use_ngrok.value, ngrok_token=ngrok_token.value, cloudflare=use_cloudflare.value, remote_moe=use_remote_moe.value, localhost_run=use_localhost_run.value, gradio=use_gradio.value)
!python /kaggle/Fooocus/launch.py --preset custom --language zh
fooocus.clear_up(False)
echo("已关闭 Fooocus")